# Homework 8

## Standard form LP barrier method
In the following three exercises, you will implement a barrier method for solving the standard
form LP
\begin{align}
  \text{minimize} & \quad c^\top x \\
  \text{subject to} & \quad Ax = b \\
  & \quad  x \succeq 0
\end{align}
with variable $x \in \mathbb{R}^n, A \in \mathbb{R}^{m \times n}, m < n$. Throughout this exercise we will
assume that A is full rank, and the sublevel sets $\{x | Ax = b, x \succeq 0, c^\top x \leq \gamma\}$ are all
bounded. (If this is not the case, the centering problem is unbounded below.)

### Centering Step
Centering step. Implement Newton’s method for solving the centering problem
\begin{align}
  \text{minimize} & \quad c^\top x - \sum_{i=1}^n \log x_i \\
  \text{subject to} & \quad Ax = b
\end{align}
with variable $x$, given a strictly feasible starting point $x_0$. <br> <br>
Your code should accept $A,b,c, \text{ and } x_0$, and return $x^*$, the primal optimal point, $\nu^*$, a dual optimal point, and the number of Newton steps executed. <br> <br>
Use the block elimination method to compute the Newton step. (You can also compute
the Newton step via the KKT system, and compare the result to the Newton step
computed via block elimination. The two steps should be close, but if any $x_i$
is very small, you might get a warning about the condition number of the KKT matrix.) <br> <br>
Plot $\lambda^2/2$ versus iteration $k$, for various problem data and initial points, to verify that
your implementation gives asymptotic quadratic convergence. As stopping criterion,
you can use $\lambda^2/2 \leq 10^{-6}$. Experiment with varying the algorithm parameters $\alpha$ and $\beta$,
observing the effect on the total number of Newton steps required, for a fixed problem
instance. Check that your computed $x^*, \nu^*$
(nearly) satisfy the KKT conditions. <br> <br>
To generate some random problem data (i.e., $A, b, c, x_0$), we recommend the following
approach. First, generate $A$ randomly. (You might want to check that it has full rank.)
Then generate a random positive vector $x_0$, and take $b = Ax_0$. (This ensures that $x_0$
is strictly feasible.) The parameter $c$ can be chosen randomly. To be sure the sublevel
sets are bounded, you can add a row to $A$ with all positive elements. If you want to
be able to repeat a run with the same problem data, be sure to set the state for the
uniform and normal random number generators.
Here are some hints that may be useful.

* We recommend computing $\lambda^2/2$ using the formula $\lambda^2 = - \Delta x_{nt}^\top \nabla f(x)$. You don’t really need $\lambda$ for anything; you can work with $\lambda^2$ instead. (This is important for reasons described below.)
* There can be small numerical errors in the Newton step $\Delta x_{nt}$ that you compute. When $x$ is nearly optimal, the computed value of $\lambda$ can actually be (slightly) negative. If you take the squareroot to get $\lambda$, you’ll get a complex number, and you’ll never recover. Moreover, your line search will never exit. However, this only happens when $x$ is nearly optimal. So if you exit on the condition $\lambda^2/2 \leq 10^{-6}$, everything will be fine, even when the computed value of $\lambda^2$ is negative.
* For the line search, you must first multiply the step size $t$ by $\beta$ until $x + t \Delta x_{nt}$ is feasible (i.e., strictly positive). If you don’t, when you evaluate $f$ you’ll be taking the logarithm of negative numbers, and you’ll never recover.


In [21]:
import numpy as np

#Problem dimensions
m = 100
n = 200

#Problem parameters
A = np.random.randn(m-1,n) * 100
A = np.r_[A,np.random.rand(n).reshape(1,-1)]
x_0 = np.random.rand(n)
b = A @ x_0
c = np.random.rand(n) * 20 + 5

b.shape, A.shape

((100,), (100, 200))

In [22]:
#Functions and derivatives
def f(x:np.array, c:np.array):
    return c.T @ x - np.sum(np.log(x))

def g(x:np.array, c:np.array):
    return c - 1/x

def H(x:np.array,c:np.array = None):
    return np.diag((x**(-2)))

def H_inv(x: np.array, c:np.array = None):
    return np.diag((x**2))

print(f"f(x) = {f(x_0,c)}")    
print(f"g(x) shape:  {(g(x_0,c)).shape}")    
print(f"H(x) shape:  {(H(x_0,c)).shape}")    

f(x) = 1781.7046045829275
g(x) shape:  (200,)
H(x) shape:  (200, 200)


In [23]:
# Backtracking parameters
alpha = 0.1
beta = 0.3

# Stopping Criterion
epsilon = 10e-6
max_iter = 1000

def backtracking_line_search(x, dx, alpha, beta):
    t = 1
    while (f(x + t * dx,c) > f(x,c) + alpha * t * g(x,c).T @ dx) or np.isnan(f(x + t * dx,c)):
        t *= beta
    return t

def newtons_centering_step(A, x_0, b, c, alpha, beta, epsilon, max_iter):
    x = x_0
    i = 0
    while i <= max_iter:
        # Compute Newton step and decrement
        w = np.linalg.solve((A @ H_inv(x) @ A.T), (-A @ H_inv(x) @ g(x, c)))
        dx_nt = - H_inv(x) @ (A.T @ w + g(x, c))
        nt_dec_2 = - dx_nt.T @ g(x, c)
        
        # Stopping Criterion
        if nt_dec_2 / 2 <= epsilon:
            break

        # Line Search
        t = backtracking_line_search(x, dx_nt, alpha, beta)
        
        # Update
        x += t * dx_nt

        # Iteration update
        i += 1

newtons_centering_step(A, x_0, b, c, alpha, beta, epsilon, max_iter)





1575.3928910349528
1499.1873781925447
1453.8168760623623
1379.0555388936
1366.1040049619637
1356.1587297101291
1350.5842031600969
1346.9929879653205
1345.1896056344317
1343.8789561158512
1343.5603612525226
1343.5550953898655


/var/folders/w5/rh2cx23174z28g3pjv50mlrr0000gn/T/ipykernel_15980/2386703098.py:3: RuntimeWarning: invalid value encountered in log
  return c.T @ x - np.sum(np.log(x))


### Feasible Start LP Solver
Using the centering code from part (1),
implement a barrier method to solve the standard form LP
\begin{align}
    \text{minimize} & \quad c^\top x \\
    \text{subject to} & \quad Ax = b \\
    & \quad x \succeq 0
\end{align}
with variable $x$, given a strictly feasible starting point $x_0$. Your LP solver should
take as argument $A, b, c,$ and $x_0$, and return $x^*$. <br> <br>
You can terminate your barrier method when the duality gap, as measured by $n/t$,
is smaller than $10^{-3}$. (If you make the tolerance much smaller, you might run into
some numerical trouble.) Check your LP solver against the solution found by cvx, for
several problem instances. <br> <br>
The comments in part (1) on how to generate random data hold here too.
Experiment with the parameter $\mu$ to see the effect on the number of Newton steps per
centering step, and the total number of Newton steps required to solve the problem. <br> <br>
Plot the progress of the algorithm, for a problem instance with $n = 500$ and $m = 100$,
showing duality gap (on a log scale) on the vertical axis, versus the cumulative total
number of Newton steps (on a linear scale) on the horizontal axis. <br> <br>
Your algorithm should return a $2 \times k$ matrix history, (where k is the total number
of centering steps), whose first row contains the number of Newton steps required
for each centering step, and whose second row shows the duality gap at the end of
each centering step. 

### LP Solver
Using the code from part (2), implement a general standard form LP
solver, that takes arguments $A, b, c$, determines (strict) feasibility, and returns an
optimal point if the problem is (strictly) feasible. <br> <br>
You will need to implement a phase I method, that determines whether the problem
is strictly feasible, and if so, finds a strictly feasible point, which can then be fed to
the code from part (2). In fact, you can use the code from part (2) to implement the
phase I method. <br> <br>
To find a strictly feasible initial point x0, we solve the phase I problem
\begin{align}
  \text{minimize} & \quad t \\
  \text{subject to} & \quad Ax =b \\
  & \quad x \succeq (1-t) \textbf{1}, \quad t \geq 0
\end{align}
with variables $x$ and $t$. If we can find a feasible $(x, t)$, with $t < 1$, then $x$ is strictly
feasible for the original problem. The converse is also true, so the original LP is strictly
feasible if and only if $t^* < 1$, where $t^*$
is the optimal value of the phase I problem. <br> <br>
We can initialize x and t for the phase I problem with any $x_0$
satisfying $Ax_0 = b$, and $t^0 = 2 - \min_i x_i^0$.(Here we can assume that $\min x_i^0 \leq 0$; otherwise $x^0$ is already a strictly feasible point, and we are done.)
You can use a change of variable $z= x + (t - 1) \textbf{1}$ to
transform the phase I problem into the form in part (2). <br> <br>
Check your LP solver against cvx on several numerical example s, including both feasible and infeasible instances.